# `Step 2: Text Embeddings`

# 0- `Config`

In [ ]:
# Path Management
import pickle
import os
# Data Handling 
import pandas as pd
import numpy as np
# Transformers 
from transformers import GPT2Model, GPT2Tokenizer
# Tokenization
import torch
from tqdm import tqdm

In [ ]:
try:
    if manager == 1:
        print("Jupyter has been running from Manager") 
except:
    temp  = "temp"
    %run s0_config.ipynb 

In [ ]:
import openai 
from openai.embeddings_utils import get_embeddings  
openai.api_key = OPENAI_API_KEY 

# 1- `Load`

In [ ]:
import_name = "subject_data"
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_setup}", f"{import_name}" + ".pkl")) , 'rb') as file:
    subject_data = pickle.load(file)   

## `Overview`

In [ ]:
#subject_data = subject_data[:200]
subject_data

In [ ]:
subject_data = subject_data.iloc[:100]

# 2- `Operation`

### 1-) `Context Construction`

In [ ]:
texts = subject_data['Cell_Type'] + " " + subject_data['Gene_Marker']
texts

### 2-) `Transformer Selection`

In [ ]:
"""
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token 

model = GPT2Model.from_pretrained('gpt2') 
"""

### 3-) `Tokenization`

In [ ]:
# Placeholder for embeddings
embeddings_list = []

for text in tqdm(texts):
    response = openai.Embedding.create(
      input=text,
      engine="text-embedding-ada-002"  # or another suitable GPT-3 model
    )
    embedding = response['data'][0]['embedding']
    embeddings_list.append(embedding)

# Convert list of embeddings to a suitable format (e.g., numpy array)
Embeddings_GPT = np.array(embeddings_list)

In [ ]:
Embeddings_GPT.shape

### 4-) `Main Data Creation`

In [ ]:
# Step 1 
data_full = subject_data.copy() 
# Step 2: 
#data_full["Cell_Type_Embeddings"] = Embeddings_BERT  
data_full["Embeddings"] = [embedding.tolist() for embedding in Embeddings_GPT]

In [ ]:
#embeddings_list = data_full["Cell_Type_Embeddings"].tolist()  # Get the column as a list of lists
#embeddings_tensor = torch.tensor(embeddings_list)             # Convert to a tensor 

# 3- `End`

In [ ]:
export_name = "data_full" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_text}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(data_full, file)   